In [2]:
!pip install optuna

In [3]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
import numpy as np

In [4]:
DEVICE = torch.device("cuda")
BATCHSIZE = 128
CLASSES = 100
DIR = os.getcwd()
EPOCHS = 100
N_TRAIN_EXAMPLES = BATCHSIZE * 300
N_VALID_EXAMPLES = BATCHSIZE * 100

In [5]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers_Conv = trial.suggest_int("n_layers_conv", 1, 4)
    n_layers_Dense = trial.suggest_int("n_layers_dense", 1, 3)
    layers = []

    #print(f"{n_layers_Conv} layers Conv and {n_layers_Dense} Dense")

    in_channel = 3
    width_height = 32

    # convolution layers
    for i in range(n_layers_Conv):
      n_filters = trial.suggest_categorical("filters_{}".format(i), [16, 32, 64, 128])
      n_kernel_size = trial.suggest_categorical("kernel_size_{}".format(i), [3, 5])
      n_stride = trial.suggest_categorical("strides_{}".format(i), [1, 2])

      # for kernel > image
      if (n_kernel_size > width_height):
        continue

      layers.append(nn.Conv2d(in_channels = in_channel, out_channels= n_filters, kernel_size= n_kernel_size, stride= n_stride,padding='valid'))
      layers.append(nn.ReLU())

      # get size of convs
      TMP = ((width_height - n_kernel_size) // n_stride) + 1
      out_shape = (TMP, TMP, n_filters)

      #print(f"in_channels = {in_channel}\nout_channels = {n_filters}\nkernel_size = {n_kernel_size}\nstride = {n_stride}\nOutput Shape = {out_shape}")
      #print('----------------------------------------------------------------')

      width_height = TMP
      in_channel = n_filters

    # Flatten layer
    layers.append(nn.Flatten())
    

    in_features = out_shape[0] * out_shape[1] * out_shape[2]
    #in_features = out_shape[0]
    for i in range(n_layers_Dense):
        out_features = trial.suggest_int("n_units_{}".format(i), 64, 128)
        layers.append(nn.Linear(in_features, out_features))

        #print(f"in_features = {in_features}\nout_features = {out_features}")
        #print('----------------------------------------------------------------')

        layers.append(nn.ReLU())

        in_features = out_features

    # last Dense layer    
    layers.append(nn.Linear(in_features, CLASSES))
    #print(f"in_features = {in_features}\nout_features = {CLASSES}")
    #print('----------------------------------------------------------------')

    #layers.append(nn.LogSoftmax(dim=1))
    layers.append(nn.Softmax(dim = 1))

    network = nn.Sequential(*layers)
    #print('----------------------------------------------------------------')
    #summary(network, (3, 32, 32))
    #print('----------------------------------------------------------------')

    return network

In [6]:
def get_CIFAR100():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

In [7]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the CIFAR100 dataset.
    train_loader, valid_loader = get_CIFAR100()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):

            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            
            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

# Q4_D
## Run 2 (without Timeout + Pruning + 100 Epoches)

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    study.optimize(objective)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-06-17 05:17:54,134] A new study created in memory with name: no-name-5021eafc-8419-4787-b2cf-0e6df389696e


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting /kaggle/working/cifar-100-python.tar.gz to /kaggle/working


[I 2022-06-17 05:35:59,188] Trial 0 finished with value: 0.0141 and parameters: {'n_layers_conv': 3, 'n_layers_dense': 3, 'filters_0': 16, 'kernel_size_0': 3, 'strides_0': 1, 'filters_1': 64, 'kernel_size_1': 3, 'strides_1': 2, 'filters_2': 32, 'kernel_size_2': 3, 'strides_2': 2, 'n_units_0': 79, 'n_units_1': 119, 'n_units_2': 128, 'optimizer': 'RMSprop', 'lr': 0.0011083750847088489}. Best is trial 0 with value: 0.0141.


Files already downloaded and verified


...

Files already downloaded and verified


[I 2022-06-17 12:16:17,161] Trial 52 pruned. 


Files already downloaded and verified


[I 2022-06-17 12:32:58,979] Trial 53 finished with value: 0.105 and parameters: {'n_layers_conv': 2, 'n_layers_dense': 1, 'filters_0': 32, 'kernel_size_0': 5, 'strides_0': 1, 'filters_1': 16, 'kernel_size_1': 5, 'strides_1': 1, 'n_units_0': 71, 'optimizer': 'Adam', 'lr': 0.00012048510754859375}. Best is trial 40 with value: 0.1576.


Files already downloaded and verified
